## Running the Supervised Optimizer

From start to finish, on pretrained weights


In [1]:
import torch
import numpy as np
import os
import json
from Hamiltonian import Ising
from model import TransformerModel
from optimizer_supervised import Optimizer

In [10]:
system_sizes = np.arange(2, 20 + 1, 2)
Hamiltonians = [Ising(L) for L in system_sizes]
data_dir_path = os.path.join("TFIM_ground_states", "2024-07-24T19-26-39.836")
for ham in Hamiltonians:
    ham.load_dataset(data_dir_path)

/home/spandan/Projects/tqs/Hamiltonian_utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  system_size = torch.tensor(system_size, dtype=torch.int64).reshape(-1)


In [11]:
# Verify that data can be retrieved from Hamiltonians
test_ham = Hamiltonians[3]  # Size 8
test_ham.dataset

,N,h,energy,state
0,8,0.50,-8.509082,"[0.6605958050461614, 0.0840743972388362, 0.084..."
1,8,0.51,-8.530118,"[0.6585782455420488, 0.085569590570777, 0.0855..."
2,8,0.52,-8.551616,"[0.6565051975177678, 0.08705266382789387, 0.08..."
3,8,0.53,-8.573583,"[0.6543751344281732, 0.08852320503071141, 0.08..."
4,8,0.54,-8.596021,"[0.6521864366727165, 0.08998077886278089, 0.08..."
...,...,...,...,...
96,8,1.46,-13.107242,"[0.25838376330919144, 0.11298188861835674, 0.1..."
97,8,1.47,-13.176419,"[0.2558137133373303, 0.11260196191599145, 0.11..."
98,8,1.48,-13.245775,"[0.2533008144129923, 0.11222628433594571, 0.11..."
99,8,1.49,-13.315306,"[0.2508435516047006, 0.11185487885467213, 0.11..."


In [12]:
energy, state = test_ham.retrieve_ground(1.48)
print(energy)
print(state)

-13.245775477442006
tensor([0.2533, 0.1122, 0.1122, 0.0931, 0.1122, 0.0544, 0.0931, 0.0878, 0.1122,
        0.0511, 0.0544, 0.0473, 0.0931, 0.0473, 0.0878, 0.0865, 0.1122, 0.0506,
        0.0511, 0.0436, 0.0544, 0.0271, 0.0473, 0.0460, 0.0931, 0.0436, 0.0473,
        0.0427, 0.0878, 0.0460, 0.0865, 0.0878, 0.1122, 0.0511, 0.0506, 0.0436,
        0.0511, 0.0256, 0.0436, 0.0427, 0.0544, 0.0256, 0.0271, 0.0246, 0.0473,
        0.0250, 0.0460, 0.0473, 0.0931, 0.0436, 0.0436, 0.0389, 0.0473, 0.0246,
        0.0427, 0.0436, 0.0878, 0.0427, 0.0460, 0.0436, 0.0865, 0.0473, 0.0878,
        0.0931, 0.1122, 0.0544, 0.0511, 0.0473, 0.0506, 0.0271, 0.0436, 0.0460,
        0.0511, 0.0256, 0.0256, 0.0250, 0.0436, 0.0246, 0.0427, 0.0473, 0.0544,
        0.0271, 0.0256, 0.0246, 0.0271, 0.0151, 0.0246, 0.0271, 0.0473, 0.0246,
        0.0250, 0.0256, 0.0460, 0.0271, 0.0473, 0.0544, 0.0931, 0.0473, 0.0436,
        0.0427, 0.0436, 0.0246, 0.0389, 0.0436, 0.0473, 0.0250, 0.0246, 0.0256,
        0.0427, 0.02

In [13]:
state.reshape(1, -1)

tensor([[0.2533, 0.1122, 0.1122, 0.0931, 0.1122, 0.0544, 0.0931, 0.0878, 0.1122,
         0.0511, 0.0544, 0.0473, 0.0931, 0.0473, 0.0878, 0.0865, 0.1122, 0.0506,
         0.0511, 0.0436, 0.0544, 0.0271, 0.0473, 0.0460, 0.0931, 0.0436, 0.0473,
         0.0427, 0.0878, 0.0460, 0.0865, 0.0878, 0.1122, 0.0511, 0.0506, 0.0436,
         0.0511, 0.0256, 0.0436, 0.0427, 0.0544, 0.0256, 0.0271, 0.0246, 0.0473,
         0.0250, 0.0460, 0.0473, 0.0931, 0.0436, 0.0436, 0.0389, 0.0473, 0.0246,
         0.0427, 0.0436, 0.0878, 0.0427, 0.0460, 0.0436, 0.0865, 0.0473, 0.0878,
         0.0931, 0.1122, 0.0544, 0.0511, 0.0473, 0.0506, 0.0271, 0.0436, 0.0460,
         0.0511, 0.0256, 0.0256, 0.0250, 0.0436, 0.0246, 0.0427, 0.0473, 0.0544,
         0.0271, 0.0256, 0.0246, 0.0271, 0.0151, 0.0246, 0.0271, 0.0473, 0.0246,
         0.0250, 0.0256, 0.0460, 0.0271, 0.0473, 0.0544, 0.0931, 0.0473, 0.0436,
         0.0427, 0.0436, 0.0246, 0.0389, 0.0436, 0.0473, 0.0250, 0.0246, 0.0256,
         0.0427, 0.0256, 0.0

In [14]:
system_sizes = np.arange(2, 20 + 1, 2).reshape(-1, 1)
Hamiltonians = [Ising(size, periodic=True) for size in system_sizes]
param_dim = Hamiltonians[0].param_dim
embedding_size = 32
n_head = 8
n_hid = embedding_size
n_layers = 8
dropout = 0
minibatch = 10000
param_range = None
point_of_interest = None
use_SR = False

Hamiltonians = [Ising(L) for L in system_sizes]
data_dir_path = os.path.join("TFIM_ground_states", "2024-07-24T19-26-39.836")
for ham in Hamiltonians:
    ham.load_dataset(data_dir_path)

print("Sizes:\n", system_sizes)
print("Hamiltonians:", Hamiltonians)
print("Dimensions of parameter space:", param_dim)
print("Number of units in a feedforward layer:", n_hid)

Sizes:
 [[4]
 [6]
 [8]]
Hamiltonians: [<Hamiltonian.Ising object at 0x7dc89ffeb770>, <Hamiltonian.Ising object at 0x7dc8a0f3fd70>, <Hamiltonian.Ising object at 0x7dc8a0e85130>]
Dimensions of parameter space: 1
Number of units in a feedforward layer: 32


/home/spandan/Projects/tqs/Hamiltonian_utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  system_size = torch.tensor(system_size, dtype=torch.int64).reshape(-1)


In [15]:
testmodel = TransformerModel(
    system_sizes,
    param_dim,
    embedding_size,
    n_head,
    n_hid,
    n_layers,
    dropout=dropout,
    minibatch=minibatch,
)

results_dir = "results"
paper_checkpoint_name = "ckpt_100000_Ising_32_8_8_0.ckpt"
paper_checkpoint_path = os.path.join(results_dir, paper_checkpoint_name)
checkpoint = torch.load(paper_checkpoint_path)
testmodel.load_state_dict(checkpoint)

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

In [16]:
opt = Optimizer(testmodel, Hamiltonians, point_of_interest=point_of_interest)

In [17]:
epochs = 1000
param_range = torch.tensor([[0.5, 1.5]])
param_step = torch.tensor([0.05])

opt.train(epochs=epochs, param_range=param_range, param_step=param_step, start_iter=0)

ValueError: Ground states not loaded yet. See load_dataset.

In [ ]:
H = Hamiltonians[2]
param = torch.tensor([0.8000])
ground_energy, psi_true = H.calc_ground(param)
psi_true

tensor([-0.4928,  0.1036,  0.1036,  ...,  0.1036,  0.1036, -0.4928],
       dtype=torch.float64)

In [ ]:
basis = H.generate_basis()
basis.shape

torch.Size([14, 16384])

In [ ]:
from model_utils import compute_psi

testmodel.set_param(system_size=H.system_size, param=param)
log_amp, log_phase = compute_psi(testmodel, basis, H.symmetry, check_duplicate=True)
print(log_amp)
print(log_phase)

tensor([-2.0506, -3.7310, -4.7868,  ..., -4.7868, -3.7310, -2.0506],
       grad_fn=<LogBackward0>)
tensor([-6.0865,  0.1709,  0.1618,  ...,  0.1618,  0.1709, -6.0865],
       grad_fn=<MulBackward0>)
